# ERP Demo

In this interactive demo, you can explore how assumptions about the timing and synchrony of neural events impact the shape and magnitude of the averaged evoked response (ERP). By manipulating factors such as onset timing, phase synchrony, and noise, you’ll gain insight into what features the ERP method is sensitive to—and which meaningful neural activity may go undetected or be misrepresented.

The ERPs in this demo are simulated as a sinusoidal waveform with an exponentially decaying envelope. While this simple model doesn’t capture all the complexities of real neural activity, it effectively highlights the mathematical assumptions underlying ERP analysis.

### Play with the parameters:

- **Number of Trials, Phase Synchrony, and Onset Synchrony**: Explore their Effects
  
- **Noise Level**: Adding Gaussian noise simulates the variability present in real EEG data. Observe how noise impacts ERP clarity, particularly when phase or onset synchrony is low, and consider what this means for real-world ERP studies.

### Guiding Questions:

- At what levels of noise and synchrony does the ERP method begin to lose sensitivity to the underlying signal?
- Which settings reveal limitations of ERP analysis, and how might these impact the interpretation of results in EEG studies?
  
This demo is designed to help you understand both the strengths and limitations of ERP analysis, which you will often encounter in the field of neuromodulation and in the broader field of cognitive neuroscience.

__Just run the following to code cells, then explore the effects of the sliders in the interactive widget!__

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets

In [15]:

# Function to generate a single ERP with exponential decay and optional Gaussian noise
def generate_erp(sampling_rate, duration, freq, phase_shift=0, jitter=0, noise_std=0):
    t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
    baseline = np.zeros(int(sampling_rate * 0.2))  # 0.2s baseline
    decay_time = t[int(sampling_rate * 0.2):]  # After 0.2s baseline
    oscillation = np.exp(-decay_time / 0.1) * np.sin(2 * np.pi * freq * decay_time + phase_shift)  # Exponentially decaying oscillation
    erp = np.concatenate([baseline, oscillation])
    
    # Apply onset jitter by shifting the ERP traces
    jitter_samples = int(np.random.uniform(-jitter, jitter) * sampling_rate)  # Convert jitter to samples
    erp = np.roll(erp, jitter_samples)  # Shift the signal by random jitter
    
    # Add Gaussian noise to ERP
    noise = np.random.normal(0, noise_std, size=erp.shape)
    erp += noise
    
    return t, erp

# Function to generate multiple ERPs with phase synchronization, onset jitter, and optional Gaussian noise
def generate_erp_set(n_traces, sampling_rate, duration, freq, phase_sync=0, jitter=0, noise_std=0):
    t, _ = generate_erp(sampling_rate, duration, freq)  # Get time vector and one ERP for baseline setup
    erps = []
    
    for i in range(n_traces):
        phase_shift = np.random.uniform(0, 2 * np.pi) * (1 - phase_sync)  # Phase shift controlled by phase_sync
        _, erp = generate_erp(sampling_rate, duration, freq, phase_shift, jitter, noise_std)
        erps.append(erp)
        
    return t, np.array(erps)

# Function to update the plot based on the sliders
def plot_erp_phase_sync(n_traces=10, freq=10, phase_sync=0, jitter=0, noise_std=0):
    # Generate ERP set with the given parameters
    t, erps = generate_erp_set(n_traces, 250, 1, freq, phase_sync, jitter, noise_std)
    
    # Calculate the average ERP
    average_erp = np.mean(erps, axis=0)
    
    # Create the plot
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))

    # Plot individual ERPs
    for erp in erps:
        axs[0].plot(t, erp, color='gray', alpha=0.7)
    axs[0].set_title("Individual ERP Traces")
    axs[0].set_xlabel("Time [s]")
    axs[0].set_ylabel("Amplitude")
    axs[0].grid(True)
    
    # Plot the average ERP
    axs[1].plot(t, average_erp, label="Average ERP", color='blue')
    axs[1].set_title("Average ERP")
    axs[1].set_xlabel("Time [s]")
    axs[1].set_ylabel("Amplitude")
    axs[1].grid(True)
    axs[1].set_ylim(-0.1, 0.15)

    plt.tight_layout()
    plt.show()

# Interactive plot with sliders, including Gaussian noise slider
interactive_plot = interactive(
    plot_erp_phase_sync,
    n_traces=widgets.IntSlider(value=10, min=10, max=100, step=10, description="N Trials:"),
    freq=widgets.FloatSlider(value=10, min=1, max=20, step=1, description="Freq(Hz):"),
    phase_sync=widgets.FloatSlider(value=0, min=0, max=1, step=0.1, description="Phase Sync"),
    jitter=widgets.FloatSlider(value=0.02, min=0, max=0.2, step=0.01, description="Onset Jitter:"),
    noise_std=widgets.FloatSlider(value=0.0, min=0, max=0.1, step=0.01, description="Noise Lvl:")
)

interactive_plot


interactive(children=(IntSlider(value=10, description='N Trials:', min=10, step=10), FloatSlider(value=10.0, d…